## Introduction
In this notebook, a simple model is trained while visualizing several graphs
* Line graphs: training loss, validation loss and accuracy as a function of epoch
* Scatter plot: Validation loss for individual validation samples
* A sample image, selectable by clicking a sample on the scatter plot 
* Bar graph: class probabilities for selected sample 

The MNIST dataset is used in this notebook to facilitate fast iteration on plot functionality updates.

While writing this notebook, the functionality developed in [03_architecture.ipynb](./03_architecture.ipynb) was expanded directly in [](../trainplotkit/subplots.py) using the workflow described in [development_workflow.ipynb](./development_workflow.ipynb)

## Imports

In [1]:
import sys, importlib
import torch
from torch import nn, optim, Tensor
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torcheval.metrics import MulticlassAccuracy

sys.path.append('..')
from trainplotkit.layout import place_subplots
from trainplotkit.plotgrid import PlotGrid
from trainplotkit.subplots.basic import TrainingCurveSP, MetricSP, ValidLossSP, ImageSP, ClassProbsSP

In [2]:
def reload_imports():
    importlib.reload(sys.modules['trainplotkit.layout'])
    importlib.reload(sys.modules['trainplotkit.plotgrid'])
    importlib.reload(sys.modules['trainplotkit.subplots.basic'])
    global place_subplots, PlotGrid, TrainingCurveSP, MetricSP, ValidLossSP, ImageSP, ClassProbsSP
    from trainplotkit.layout import place_subplots
    from trainplotkit.plotgrid import PlotGrid
    from trainplotkit.subplots.basic import TrainingCurveSP, MetricSP, ValidLossSP, ImageSP, ClassProbsSP

## Data preparation
Based on [PyTorch Basic MNIST Example](https://github.com/pytorch/examples/blob/main/mnist/main.py)

In [3]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True, transform=transform)
dataset2 = datasets.MNIST('../data', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=64, num_workers=1, pin_memory=True, shuffle=True)
test_loader  = torch.utils.data.DataLoader(dataset2, batch_size=500, num_workers=1, pin_memory=True, shuffle=False)

In [4]:
dataset1[0][0].shape

torch.Size([1, 28, 28])

## Model
Based on [PyTorch Basic MNIST Example](https://github.com/pytorch/examples/blob/main/mnist/main.py)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [6]:
class SmallNet(nn.Module):
    """Smaller model to iterate faster while developing plots"""
    def __init__(self):
        super(SmallNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, 2)
        self.conv2 = nn.Conv2d(8, 8, 3, 2)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(72, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

## Training loop
* Based on [PyTorch Basic MNIST Example](https://github.com/pytorch/examples/blob/main/mnist/main.py)
* Small edits to add calls to `PlotGrid` methods

In [7]:
def train_epoch(model, device, train_loader, optimizer, pg:PlotGrid):
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pg.after_batch(training=True, inputs=data, targets=target, predictions=output, loss=loss)
    pg.after_epoch(training=True)

def test_epoch(model, device, test_loader, pg:PlotGrid):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = F.nll_loss(output, target)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            pg.after_batch(training=False, inputs=data, targets=target, predictions=output, loss=loss)
    pg.after_epoch(training=False)

def fit(n_epochs, model, device, train_loader, test_loader, optimizer, scheduler, pg:PlotGrid):
    for _ in range(n_epochs):
        train_epoch(model, device, train_loader, optimizer, pg)
        test_epoch(model, device, test_loader, pg)
        scheduler.step()
    pg.after_fit()

In [8]:
logits = Tensor([[-1,-2,-3,-4],[5,6,7,8],[3,4,5,6]])
torch.softmax(logits, dim=1)

tensor([[0.6439, 0.2369, 0.0871, 0.0321],
        [0.0321, 0.0871, 0.2369, 0.6439],
        [0.0321, 0.0871, 0.2369, 0.6439]])

## Define plots and run training loop

In [9]:
reload_imports()

In [10]:
device = 'cuda'
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1.0)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

# Subplots
batch_loss_fn = lambda preds,targs: F.nll_loss(preds,targs,reduction='none')
probs_fn = lambda preds: torch.softmax(preds, dim=1)
sps = [
    TrainingCurveSP(colspan=2), 
    ValidLossSP(batch_loss_fn, remember_past_epochs=True, colspan=2), 
    ImageSP(dataset2, rowspan=2),
    MetricSP("Accuracy", MulticlassAccuracy(), colspan=2), 
    ClassProbsSP(probs_fn, remember_past_epochs=True, class_names=[f"{i}" for i in range(10)], colspan=2),
]
pg = PlotGrid(num_grid_cols=5, subplots=sps)
pg.show()

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Training loss',
              'type': 'scatter',
              'uid': '4a78255f-7f31-4dac-bb11-2a9f76f5555e',
              'x': [],
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'mode': 'lines+markers',
              'name': 'Validation loss',
              'type': 'scatter',
              'uid': '54f09261-56d6-49dd-adf8-3b88290c498c',
              'x': [],
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'hoverinfo': 'skip',
              'marker': {'color': 'rgba(0,0,0,0.2)', 'line': {'color': 'black', 'width': 2}},
              'mode': 'markers',
              'showlegend': False,
              'type': 'scatter',
              'uid': 'aa14356b-d5d6-4997-9c5f-899733d75c23',
              'x': [],
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'mode': 'markers',
      

In [11]:
n_epochs = 5
fit(n_epochs, model, device, train_loader, test_loader, optimizer, scheduler, pg)

### Saving a static copy
When done interacting with the figure, add a static copy to the notebook DOM for sharing purposes 

In [12]:
pg.show_static()

### Some diagnostics
* Two issues were discovered by recalculating predictions and loss for individual samples
  * Initially, some of the subplots forgot to call `.detach()`
  * Initially, the validation set was shuffled after each epoch, causing strange behavior on the `ValidLossSP` and `ClassProbsSP` subplots 

In [51]:
if False:
    it = iter(test_loader)
    data, target = next(it)
    data.shape, target.shape, [int(i) for i in target]

In [52]:
data = torch.cat([dataset2[i][0][None] for i in range(20)], dim=0)
target = Tensor([dataset2[i][1] for i in range(20)]).to(torch.int64)
data.shape, target.shape, [int(i) for i in target]


(torch.Size([20, 1, 28, 28]),
 torch.Size([20]),
 [7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4])

In [53]:
with torch.no_grad():
    data, target = data.to(device), target.to(device)
    output = model(data)
    loss = F.nll_loss(output, target, reduction='none')

In [54]:
idx=0
target[idx], output[idx:idx+1], torch.softmax(output[idx:idx+1], dim=1), loss[idx]

(tensor(7, device='cuda:0'),
 tensor([[-23.5344, -19.8993, -19.8058, -17.7781, -23.5802, -25.1682, -29.4595,
            0.0000, -24.0103, -17.4187]], device='cuda:0'),
 tensor([[6.0140e-11, 2.2794e-09, 2.5031e-09, 1.9014e-08, 5.7447e-11, 1.1738e-11,
          1.6066e-13, 1.0000e+00, 3.7364e-11, 2.7238e-08]], device='cuda:0'),
 tensor(-0., device='cuda:0'))

In [55]:
sps[4].probs[-1][idx]

[6.013967795981401e-11,
 2.2794446330465234e-09,
 2.5030506556333876e-09,
 1.901412360894028e-08,
 5.7446675888472853e-11,
 1.1737682874268796e-11,
 1.606627699803076e-13,
 1.0,
 3.736366077644604e-11,
 2.7237721411665916e-08]

### Updating subplot titles
Subplot title annotations are created and positioned upon subplot creation using the `subplot_titles` parameter passed to `make_subplots`. This was a quick experiment to see how these titles may be updated later in response to user interactions and how multi-line titles are displayed

In [56]:
pg.widget.layout.annotations[2].text = 'Input: sample 0<br>Target=7'